# 09 향후 판매량 예측

## 9.3 베이스라인 모델

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
# 데이터 블로오기
data_path = '../../data/09_sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 9.3.1 피처 엔지니어링 I: 피처명 한글화

### 9.3.2 피처 엔지니어링 II: 데이터 다운캐스팅

### 9.3.3 피처 엔지니어링 III: 데이터 조합 생성

### 9.3.4 피처 엔지니어링 IV: 타깃값(월간 판매량) 추가

### 9.3.5 피처 엔지니어링 V: 테스트 데이터 이어붙이기

### 9.3.6 피처 엔지니어링 VI: 나머지 데이터 병합(최종 데이터 생성)

### 9.3.7 피처 엔지니어링 VII: 마무리

### 9.3.8 모델 훈련 및 성능 검증

### 9.3.9 예측 및 결과 제출